In [38]:
import asammdf
import hashlib
import get_files
import pony.orm
from pony.orm.core import EntityMeta
from pony import orm
import py
import time

pony.orm.set_sql_debug(False)

In [33]:
db = pony.orm.Database()
db.bind(
    provider='sqlite',
    filename=':memory:',
)

class MDF(db.Entity):
    name = pony.orm.Required(
        str,
    )
    version = pony.orm.Required(
        str,
    )
    sha256 = pony.orm.Required(
        str, 
    )
    size = pony.orm.Optional(
        int, 
    )
    size_mb = pony.orm.Optional(
        float, 
    )
    atime = pony.orm.Optional(
        float,
    )
    channels = pony.orm.Set(
        'Channel',
    )
    
class Channel(db.Entity):
    name = pony.orm.Required(
        str,
        unique=True,
    )
    mdfs = pony.orm.Set(
        "MDF",
    )
db.generate_mapping(create_tables=True)

In [34]:
def upsert(cls, get, set=None):
    """
    Interacting with Pony entities.

    :param cls: The actual entity class
    :param get: Identify the object (e.g. row) with this dictionary
    :param set: 
    :return:
    """
    # does the object exist
    assert isinstance(cls, EntityMeta), "{cls} is not a database entity".format(cls=cls)

    # if no set dictionary has been specified
    set = set or {}

    if not cls.exists(**get):
        # make new object
        return cls(**set, **get)
    else:
        # get the existing object
        obj = cls.get(**get)
        for key, value in set.items():
            obj.__setattr__(key, value)
        return obj

In [35]:
data_files = get_files.get_files(
    directory="../Data/",
    extensions=[".mdf", ".mf4"],
)

In [36]:
def index_data_file(data_file):
    data_file_ = py.path.local(
        path=data_file,
    )
    
    mdf = asammdf.MDF(data_file)

    channels = list()
    mdf.channels_db.keys()
    for channel in mdf.channels_db.keys():
        channel_ = upsert(Channel, {"name": channel})
        channels.append(channel_)

    sha256 = data_file_.computehash(
        hashtype="sha256",
    )
        
    _ = upsert(
        cls=MDF,
        get={"sha256": sha256},
        set={
            "name": data_file,
            "version": mdf.version,
            "channels": channels,
        },
    )

    db.commit()

In [ ]:
for data_file in data_files:
    break

index_data_file(
    data_file=data_file,
)

In [10]:
t1=t
for data_file in data_files:
    index_data_file(
        data_file=data_file,
    )

In [14]:
data_file_ = py.path.local(data_file)

In [17]:
data_file_.size()/1024 ** 2

0.9199237823486328

In [27]:
data_file_.atime()

1518540509.4020863

In [24]:
data_file_.computehash(
    hashtype="sha256"
)

'af8cbea3fc7e29d6ee2dafc8b7d2f9dfd32bc75a2266ce621ddd07fdd4596d72'